In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import scipy.stats
import numpy as np
import scipy.stats
import pandas as pd

In [2]:
#Modificado

boston = datasets.load_boston()

X = pd.DataFrame(data=boston.data, columns = boston['feature_names']) 
y = pd.DataFrame(boston.target)

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [3]:
#Modificado

# correlation_filtering()

# se define la funcion
def correlation_filtering(data_frame, treshold):

    df = pd.DataFrame(data_frame)
    # Create correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # guarda las columnas que se van a tirar
    to_drop =[]

    for n in range(df.shape[1]): # itera sobre columnas del data frame
        
        if any(upper.iloc[:,n]>treshold):     # si algun elemento de la matriz de correlacion de la columna n es mayor al treshold
            print(upper.columns[n], " Sobrepasa el umbral")
            to_drop.append(df.columns[n])     # se agrega para borrarse
            
    print('columns dropped: ', to_drop)            # columnas que se van a tirar   

    # Drop features 
    df.drop(columns= to_drop,inplace=True)

    return df

In [4]:
fcf = correlation_filtering(X_train, 0.7)
fcf

NOX  Sobrepasa el umbral
AGE  Sobrepasa el umbral
DIS  Sobrepasa el umbral
TAX  Sobrepasa el umbral
columns dropped:  ['NOX', 'AGE', 'DIS', 'TAX']


,CRIM,ZN,INDUS,CHAS,RM,RAD,PTRATIO,B,LSTAT
81,0.04462,25.0,4.86,0.0,6.619,4.0,19.0,395.63,7.22
122,0.09299,0.0,25.65,0.0,5.961,2.0,19.1,378.09,17.93
162,1.83377,0.0,19.58,1.0,7.802,5.0,14.7,389.61,1.92
370,6.53876,0.0,18.10,1.0,7.016,24.0,20.2,392.05,2.96
263,0.82526,20.0,3.97,0.0,7.327,5.0,13.0,393.42,11.25
...,...,...,...,...,...,...,...,...,...
155,3.53501,0.0,19.58,1.0,6.152,5.0,14.7,88.01,15.02
301,0.03537,34.0,6.09,0.0,6.590,7.0,16.1,395.75,9.50
79,0.08387,0.0,12.83,0.0,5.874,5.0,18.7,396.06,9.10
360,4.54192,0.0,18.10,0.0,6.398,24.0,20.2,374.56,7.79


In [5]:
boston = datasets.load_boston()

X = boston.data
y = boston.target

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [6]:
# Fast Correlation Based filtering v2

dfx = pd.DataFrame(X) # base que NO incluye la variable objetivo
 
# 1. Obtener la correlación de cada variable en X con la variable de salida y guardar las que pasen el umbral
treshold = 0.5
corr_xy = [] # aquí se van a guardar las correlaciones de cada xi con y
high_corr = [] # aquí se guardan las variables que están por arriba del umbral

#Lista con las caractrísticas xi más relacionadas con y
for n in range(dfx.shape[1]): # itera por columnas del data frame
    column = dfx.iloc[:,n]    # guarda la columna
    a = abs(np.corrcoef(column, y)[0,1]) # calcula la correlacion en valor absoluto
    if a > treshold:                     
        high_corr.append(dfx.columns[n])
max = len(high_corr) # Número de veces que iterará la función

for iteracion in range(max):
    
    #print("ITERACION: ",iteracion)
    
    corr_xy = []

    #Lista de correlaciones de todas las caractrísticas xi con y
    for n in range(dfx.shape[1]): # itera por columnas del data frame
        column = dfx.iloc[:,n]    # guarda la columna
        a = abs(np.corrcoef(column, y)[0,1]) # calcula la correlacion en valor absoluto
        corr_xy.append(a)                

    #print("corr_xy: ",corr_xy) 
    #print("high_corr: ",high_corr)

    # 2. Ordenar las variables de mayor a menor correlación con Y
    temp = pd.DataFrame(corr_xy).reset_index()
    temp = temp.rename(columns={0: 'corr'})
    temp = temp.sort_values(by=['corr'], ascending=False)
    #print("temp: ",temp)  

    #3. Medir la correlación de la mejor variable con las demás
    feature = temp.iloc[0,0] # Característica xi más correlacionado con y
    df_xi = dfx.iloc[:,feature] # Dataframe con el xi más correlacionado con y  
    df_witouth_xi = pd.DataFrame(dfx)
    df_witouth_xi.drop(columns = dfx.columns[feature] ,inplace=True) # Dataframe sin el xi más correlacionado con y

    corr_xi_all = abs(df_witouth_xi.corrwith(df_xi)) # Correlacion de xi con las demás características
    #print("Corr_xi_all", corr_xi_all)

    # 4. Tirar variables que estén muy correlacionadas con la primera variable.
    to_drop =[] # vector de variables que se van a borrar

    for n in range(df_witouth_xi.shape[1]): # itera sobre columnas del data frame

            if (corr_xi_all.iloc[n]>treshold):     # si algun elemento de la matriz de correlacion de la columna n es mayor al treshold
                #print(corr_xi_all.iloc[n], " sobrepasa el umbral")
                to_drop.append(dfx.columns[n])     # se agrega para borrarse
            else:
                (corr_xi_all.iloc[n], " no sobrepasa el umbral")

    #print('columns dropped: ', to_drop)            # columnas que se van a tirar   

    # Drop features 
    df_witouth_xi.drop(columns= to_drop,inplace=True)

    # 5. Dataframe de Características
  
    if iteracion == 0:
        features_df = pd.DataFrame(df_xi)
    elif (iteracion > 0) and (iteracion < max-1):
        features_df = pd.concat([features_df, df_xi], axis=1, sort=False)
    else:
        features_df = pd.concat([features_df, df_witouth_xi], axis=1, sort=False)

    dfx = pd.DataFrame(df_witouth_xi) #Dataframe que se vuelve a evaluar
    #print("features df: ",features_df)
    #print("df_withouth feature: ",df_witouth_xi)
    #print("df_nuevo: ",dfx)
    
print("features df: ",features_df)

features df:         12    10    1    3    8       11
0    4.98  15.3  18.0  0.0  1.0  396.90
1    9.14  17.8   0.0  0.0  2.0  396.90
2    4.03  17.8   0.0  0.0  2.0  392.83
3    2.94  18.7   0.0  0.0  3.0  394.63
4    5.33  18.7   0.0  0.0  3.0  396.90
..    ...   ...   ...  ...  ...     ...
501  9.67  21.0   0.0  0.0  1.0  391.99
502  9.08  21.0   0.0  0.0  1.0  396.90
503  5.64  21.0   0.0  0.0  1.0  396.90
504  6.48  21.0   0.0  0.0  1.0  393.45
505  7.88  21.0   0.0  0.0  1.0  396.90

[506 rows x 6 columns]
